In [153]:
import pandas as pd
import sklearn
import plotly.graph_objects as go
import urllib, json

In [219]:
beer = pd.read_csv("Beer.csv")
locations_beer = pd.read_csv("Location_Breweries.csv")
cocktails = pd.read_csv("Cocktails.csv")

locations_beer.columns = ['brewery_name', 'geometry_location_lat', 'geometry_location_lng', 'location']

In [220]:
new_beer = beer['brewery_name'].str.split(" ", n = 2, expand = True) 
new_location = locations_beer['brewery_name'].str.split(" ", n = 2, expand = True) 


beer = beer.merge(new_beer, left_index=True, right_index=True)

locations_beer = locations_beer.merge(new_location, left_index=True, right_index=True)


In [249]:
beer

,id,nameDisplay,abv,isOrganic,styleId,style_categoryId,style_category_name,style_name,style_shortName,style_description,...,brewery_isInBusiness,brewery_isVerified,description,originalGravity,ibu,style_ogMax,style_category_description,0,1,2
0,c4f2KE,'Murican Pilsner,5.5,N,98.0,8.0,North American Lager,American-Style Pilsener,American Pilsener,This classic and unique pre-Prohibition Americ...,...,Y,N,NaN,NaN,NaN,NaN,NaN,Sierra,Nevada,Brewing Company
1,zTTWa2,11.5° PLATO,4.5,N,164.0,3.0,North American Origin Ales,Session India Pale Ale,Session IPA,Session India Pale Ales are gold to copper. Ch...,...,Y,N,The Plato scale is a measurement of the densit...,1.046,35.0,NaN,NaN,Sierra,Nevada,Brewing Company
2,zfP2fK,12th Of Never,5.5,N,25.0,3.0,North American Origin Ales,American-Style Pale Ale,American Pale,American pale ales range from deep golden to c...,...,Y,N,"Tropically Hoppy. Light, yet Full-Bodied. Brig...",1.050,45.0,NaN,NaN,Lagunitas,Brewing,Company
3,xwYSL2,15th Anniversary Ale,NaN,N,5.0,1.0,British Origin Ales,Extra Special Bitter,ESB,Extra special bitter possesses medium to stron...,...,Y,N,For the ﬁrst ever SweetWater anniversary beer ...,NaN,NaN,NaN,NaN,SweetWater,Brewing,Company
4,UJGpVS,16 So Fine Red Wheat Wine,11.0,N,35.0,3.0,North American Origin Ales,American-Style Wheat Wine Ale,Wheatwine,American style wheat wines range from gold to ...,...,Y,N,For our super heady 16 year anniversary beer w...,NaN,NaN,NaN,NaN,SweetWater,Brewing,Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,CScss3,Yonder Bock - Beer Camp Across America (2014),7.7,N,89.0,7.0,European-germanic Lager,German-Style Heller Bock/Maibock,Maibock,"The German word helle means light colored, and...",...,Y,N,Tampa is a long way from our home base in Chic...,1.078,45.0,NaN,NaN,Sierra,Nevada,Brewing Company
1105,a8u4ic,Yvan The Great - Beer Camp Across America (2014),6.3,N,61.0,5.0,Belgian And French Origin Ales,Belgian-Style Blonde Ale,Belgian Blonde,Belgian-style blond ales are characterized by ...,...,Y,N,"As longtime friends, Russian River brewmaster ...",1.053,50.0,NaN,NaN,Sierra,Nevada,Brewing Company
1106,9GBACV,Zepher Ale,7.5,N,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,N,NaN,NaN,NaN,NaN,NaN,Lagunitas,Brewing,Company
1107,nwNOxm,ZiegenBock Amber,4.9,N,101.0,8.0,North American Lager,American-Style Amber Lager,American Amber Lager,American-style amber lagers are light amber to...,...,Y,N,NaN,NaN,NaN,NaN,NaN,Anheuser-Busch,InBev,None


In [224]:
locations_beer.head()

,brewery_name,geometry_location_lat,geometry_location_lng,location,0,1,2
0,Wynkoop Brewing Company,39.753454,-104.998467,Denver,Wynkoop,Brewing,Company
1,Epic Brewing Company,39.763171,-104.981361,Denver,Epic,Brewing,Company
2,Cerebral Brewing,39.739747,-104.945446,Denver,Cerebral,Brewing,None
3,Alpine Dog Brewing Company,39.740326,-104.974929,Denver,Alpine,Dog,Brewing Company
4,Baere Brewing Company,39.721407,-104.987007,Denver,Baere,Brewing,Company


In [ ]:
ml_df = beer[['nameDisplay', 'abv', 'ibu', 'originalGravity',
              'style_name', 'style_category_name', 'style_ibuMin', 
              'style_ibuMax', 'style_abvMin', 'style_abvMax', 
              'style_srmMin', 'style_srmMax', 'style_ogMin', 
              'style_ogMax', 'style_fgMin', 'style_fgMax']]


In [ ]:
ml_df['new_og'] = ml_df[['originalGravity', 'style_ogMin', 'style_ogMax']].mean(axis=1) 
ml_df['new_ibu'] = ml_df[['ibu', 'style_ibuMin', 'style_ibuMax']].mean(axis=1) 
ml_df['new_abv'] = ml_df[['abv', 'style_abvMin', 'style_abvMax']].mean(axis=1) 


In [ ]:
ml_df.head()

In [ ]:
ml_df_filtered = ml_df[ml_df['new_og'].notna() | ml_df['originalGravity'].notna() | ml_df['style_ogMin'].notna() | ml_df['style_ogMax'].notna()]

ml_df_filtered = ml_df_filtered[ml_df_filtered['new_ibu'].notna() | ml_df_filtered['ibu'].notna() | ml_df_filtered['style_ibuMin'].notna() | ml_df_filtered['style_ibuMax'].notna()]

ml_df_filtered = ml_df_filtered[ml_df_filtered['new_abv'].notna() | ml_df_filtered['abv'].notna() | ml_df_filtered['style_abvMin'].notna() | ml_df_filtered['style_abvMax'].notna()]

ml_df_filtered['abv'] = ml_df_filtered['abv'].fillna(ml_df_filtered['new_abv'])
ml_df_filtered['originalGravity'] = ml_df_filtered['originalGravity'].fillna(ml_df_filtered['new_og'])
ml_df_filtered['ibu'] = ml_df_filtered['ibu'].fillna(ml_df_filtered['new_ibu'])


In [ ]:
ml_df_filtered = ml_df_filtered[['nameDisplay', 'abv', 'ibu', 'originalGravity',
                                 'style_name', 'style_category_name', 
                                 'style_srmMin', 'style_srmMax', 'style_fgMin', 'style_fgMax',]]



In [ ]:
ml_df_filtered = ml_df_filtered.dropna()

In [ ]:
ml_df_filtered = ml_df_filtered.reset_index(drop=True)

In [ ]:
ml_df_filtered.head()

In [ ]:
ml_df_filtered.to_csv("Beer_Analysis.csv", index=False)

In [466]:
#left = beer
#node 1 = category
#node 2 = brewery
#right = location

beer_sankey = beer[['nameDisplay', 'abv', 'ibu', 'originalGravity', 'style_name', 'style_category_name', 
                    'brewery_name']]

beer_sankey['value'] = 1
beer_sankey['style_value'] = 1109
beer_sankey['style_category_value'] = 1233

#################################################################

temp = beer_sankey.style_name.value_counts().index.tolist()

for category in temp:
    beer_sankey.loc[beer_sankey['style_name'] == category, 'style_value'] += temp.index(category)

#################################################################
    
temp = beer_sankey.style_category_name.value_counts().index.tolist()

for category in temp:
    beer_sankey.loc[beer_sankey['style_category_name'] == category, 'style_category_value'] += temp.index(category)

#################################################################

labels = beer_sankey.nameDisplay.tolist()

temp = beer_sankey.style_name.value_counts().index.tolist()
for label in temp:
    labels.append(label)

temp = beer_sankey.style_category_name.value_counts().index.tolist()
for label in temp:
    labels.append(label)
      
#################################################################

source = beer_sankey.index.values.tolist()
temp = [*range(1109,1109+124)]
for index in temp:
    source.append(index)

#################################################################  

target = beer_sankey.style_value.tolist()
temp = beer_sankey.groupby('style_value')['style_category_value'].mean().tolist()

for category in temp:
    target.append(category)

#################################################################  

value = beer_sankey.value.tolist()

temp = beer_sankey.style_name.value_counts().tolist()
for count in temp:
    value.append(count)

value_dict = {'labels':labels,'source':source,'target':target,'value':value}

with open('beer_sankey.json', 'w') as fp:
    json.dump(value_dict, fp)
    


/Users/richagautam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/richagautam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/richagautam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [467]:
#################################################################  

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labels,
        color = "blue"
    ),
    link = dict(
        source = source,
        target = target,
        value = value
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()


In [486]:
cocktail_sankey = cocktails[['strDrink', 'strIngredient1', 'strIngredient2', 
                             'strIngredient3', 'strIngredient4', 'strIngredient5', 
                             'strIngredient6', 'strIngredient7', 'strIngredient8', 
                             'strIngredient9', 'strIngredient10', 'strIngredient11',
                             'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15']]

cocktail_sankey = cocktail_sankey.drop_duplicates()

In [487]:
cocktail_sankey = pd.melt(cocktail_sankey, id_vars=['strDrink'], value_vars=['strIngredient1', 'strIngredient2', 
                             'strIngredient3', 'strIngredient4', 'strIngredient5', 
                             'strIngredient6', 'strIngredient7', 'strIngredient8', 
                             'strIngredient9', 'strIngredient10', 'strIngredient11',
                             'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15'])

In [488]:
cocktail_sankey = cocktail_sankey.dropna()

In [490]:
cocktail_sankey.columns = ['drink', 'ingredient_number', 'ingredient']
cocktail_sankey

,drink,ingredient_number,ingredient
0,'57 Chevy with a White License Plate,strIngredient1,Creme de Cacao
1,155 Belmont,strIngredient1,Dark rum
2,501 Blue,strIngredient1,Blue Curacao
3,747 Drink,strIngredient1,Vodka
4,A Gilligan's Island,strIngredient1,Vodka
...,...,...,...
2258,Radioactive Long Island Iced Tea,strIngredient8,Malibu rum
2275,3-Mile Long Island Iced Tea,strIngredient8,Bitters
2396,151 Florida Bushwacker,strIngredient8,Vanilla ice-cream
2520,Arizona Twister,strIngredient9,Pineapple


In [513]:
#left = ingredients
#right = cocktail

cocktail_sankey['value'] = 1
cocktail_sankey['source_value'] = 0
cocktail_sankey['target_value'] = 0

#################################################################

labels = cocktail_sankey.ingredient.value_counts().index.tolist()
for label in labels:
    cocktail_sankey.loc[(cocktail_sankey['ingredient'] == label), 'source_value'] += labels.index(label)

temp = cocktail_sankey.drink.value_counts().index.tolist()
for label in temp:
    labels.append(label)

for label in temp:
    cocktail_sankey.loc[(cocktail_sankey['drink'] == label), 'target_value'] += labels.index(label)

#################################################################

source = cocktail_sankey.source_value.tolist()

#################################################################  

target = cocktail_sankey.target_value.tolist()

#################################################################  

value = cocktail_sankey.value.tolist()

value_dict = {'labels':labels,'source':source,'target':target,'value':value}

with open('cocktail_sankey.json', 'w') as fp:
    json.dump(value_dict, fp)
    

In [514]:
#################################################################  

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labels,
        color = "blue"
    ),
    link = dict(
        source = source,
        target = target,
        value = value
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()
